In [1]:
import random
import numpy as np
import numpy.matlib
import pandas as pd

feature_i = pd.read_csv("p2/feature_i.csv", index_col=0)
feature_i = feature_i.loc[feature_i["day"]>=28,].reset_index()
del feature_i["index"]
feature_iv = pd.read_csv("p2/feature_iv.csv", index_col=0).reset_index()
del feature_iv["index"]
new_feature = pd.merge(feature_i, feature_iv)
new_feature

,pluno,bndno,type5,type4,type3,type2,day,weekday,qty,d-1,...,d-7,ave-w-2,ave-w-3,ave-w-4,max-w-2,max-w-3,max-w-4,min-w-2,min-w-3,min-w-4
0,22000005,22000005,22000,2200,220,22,28,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,22000005,22000005,22000,2200,220,22,29,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,22000005,22000005,22000,2200,220,22,30,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,22000005,22000005,22000,2200,220,22,31,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,22000005,22000005,22000,2200,220,22,32,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93935,27421002,27421002,27421,2742,274,27,177,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93936,27421002,27421002,27421,2742,274,27,178,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93937,27421002,27421002,27421,2742,274,27,179,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93938,27421002,27421002,27421,2742,274,27,180,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [2]:
day = 18*7+28-1

In [3]:
feature_name = ["pluno", "bndno", "type5", "type4", "type3", "type2", "day", "weekday", "d-1", "d-2", "d-3", "d-4", "d-5", "d-6", "d-7",
               "ave-w-2","ave-w-3","ave-w-4","max-w-2","max-w-3","max-w-4","min-w-2","min-w-3","min-w-4"]
x_i = new_feature.loc[(new_feature["day"] <= day),feature_name].values
y_i = new_feature.loc[(new_feature["day"] <= day),"qty"].values
x_i = np.reshape(x_i, (-1, 15+9))
y_i = np.reshape(y_i, (-1, 1))

from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X_i = sc_X.fit_transform(x_i)
Y_i = sc_y.fit_transform(y_i)

In [4]:
import time
time_list = [[0,0],[0,0],[0,0]]

from sklearn.svm import SVR
time_start=time.time()
regressor_i = SVR(kernel = 'rbf')
regressor_i.fit(X_i, Y_i)
time_end=time.time()
time_list[0][0] = time_end - time_start

test_i = new_feature.loc[(new_feature["day"] > day-28),:].reset_index()
test_i["predict"] = test_i["qty"]
time_start=time.time()
for i, row in test_i.iterrows():
    if row["day"] <= day:
        continue
    j = 1
    while j <= row["day"]%7:
        test_i.loc[i, "d-"+str(j)] = test_i.loc[i-j, "predict"]
        j = j+1
#     week_2 = test_i[i-14:i-8]["predict"].values
#     week_3 = test_i[i-21:i-15]["predict"].values
#     week_4 = test_i[i-28:i-22]["predict"].values
#     test_i.loc[i,"ave-w-2"] = np.mean(week_2)
#     test_i.loc[i,"min-w-2"] = min(week_2)
#     test_i.loc[i,"max-w-2"] = max(week_2)
#     test_i.loc[i,"ave-w-3"] = np.mean(week_3)
#     test_i.loc[i,"min-w-3"] = min(week_3)
#     test_i.loc[i,"max-w-3"] = max(week_3)
#     test_i.loc[i,"ave-w-4"] = np.mean(week_4)
#     test_i.loc[i,"min-w-4"] = min(week_4)
#     test_i.loc[i,"max-w-4"] = max(week_4)
    y_pred = regressor_i.predict(
        sc_X.transform(
            np.reshape(
            row[feature_name].values
            , (-1, len(feature_name)))))
    test_i.loc[i, "predict"] = sc_y.inverse_transform(y_pred)
test_i
time_end=time.time()
time_list[0][1] = time_end - time_start

C:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [5]:
# i. random forest
from sklearn.ensemble import RandomForestRegressor 
time_start=time.time()
rfr_i =RandomForestRegressor()
rfr_i.fit(X_i, Y_i)
time_end=time.time()
time_list[1][0] = time_end - time_start

test_i_rfr = new_feature.loc[(new_feature["day"] > day-28),:].reset_index()
test_i_rfr["predict"] = test_i_rfr["qty"]
time_start=time.time()
for i, row in test_i_rfr.iterrows():
    if row["day"] <= day:
        continue
    j = 1
    while j <= row["day"]%7:
        test_i_rfr.loc[i, "d-"+str(j)] = test_i_rfr.loc[i-j, "predict"]
        j = j+1
#     week_2 = test_i_rfr[i-14:i-8]["predict"].values
#     week_3 = test_i_rfr[i-21:j-15]["predict"].values
#     week_4 = test_i_rfr[j-28:j-22]["predict"].values
#     test_i_rfr.loc[i,"ave-w-2"] = np.mean(week_2)
#     test_i_rfr.loc[i,"min-w-2"] = min(week_2)
#     test_i_rfr.loc[i,"max-w-2"] = max(week_2)
#     test_i_rfr.loc[i,"ave-w-3"] = np.mean(week_3)
#     test_i_rfr.loc[i,"min-w-3"] = min(week_3)
#     test_i_rfr.loc[i,"max-w-3"] = max(week_3)
#     test_i_rfr.loc[i,"ave-w-4"] = np.mean(week_4)
#     test_i_rfr.loc[i,"min-w-4"] = min(week_4)
#     test_i_rfr.loc[i,"max-w-4"] = max(week_4)
    y_pred = rfr_i.predict(
        sc_X.transform(
            np.reshape(
            row[feature_name].values
            , (-1, len(feature_name)))))
    test_i_rfr.loc[i, "predict"] = sc_y.inverse_transform(y_pred)
test_i_rfr
time_end=time.time()
time_list[1][1] = time_end - time_start

C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


In [6]:
# i. MLP
from sklearn.neural_network import MLPRegressor
time_start=time.time()
mlp_i = MLPRegressor(
        hidden_layer_sizes=(100,50), activation='relu',solver='adam',
        alpha=0.001,max_iter=200)
# mlp_i = MLPRegressor()
mlp_i.fit(X_i,Y_i)
time_end=time.time()
time_list[2][0] = time_end - time_start

test_i_mlp = new_feature.loc[(new_feature["day"] > day-28),:].reset_index()
test_i_mlp["predict"] = test_i_mlp["qty"]
time_start=time.time()
for i, row in test_i_mlp.iterrows():
    if row["day"] <= day:
        continue
    j = 1
    while j <= row["day"]%7:
        test_i_mlp.loc[i, "d-"+str(j)] = test_i_mlp.loc[i-j, "predict"]
        j = j+1
#     week_2 = test_i_mlp[i-14:i-8]["predict"].values
#     week_3 = test_i_mlp[i-21:j-15]["predict"].values
#     week_4 = test_i_mlp[j-28:j-22]["predict"].values
#     test_i_mlp.loc[i,"ave-w-2"] = np.mean(week_2)
#     test_i_mlp.loc[i,"min-w-2"] = min(week_2)
#     test_i_mlp.loc[i,"max-w-2"] = max(week_2)
#     test_i_mlp.loc[i,"ave-w-3"] = np.mean(week_3)
#     test_i_mlp.loc[i,"min-w-3"] = min(week_3)
#     test_i_mlp.loc[i,"max-w-3"] = max(week_3)
#     test_i_mlp.loc[i,"ave-w-4"] = np.mean(week_4)
#     test_i_mlp.loc[i,"min-w-4"] = min(week_4)
#     test_i_mlp.loc[i,"max-w-4"] = max(week_4)
    y_pred = mlp_i.predict(
        sc_X.transform(
            np.reshape(
            row[feature_name].values
            , (-1, len(feature_name)))))
    test_i_mlp.loc[i, "predict"] = sc_y.inverse_transform(y_pred)
test_i_mlp

time_end=time.time()
time_list[2][1] = time_end - time_start

C:\Users\admin\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
import math

def getRSE(data):
    avg = np.mean(data["qty"])
    return math.sqrt((data["qty"]-data["predict"]).apply(lambda x:math.pow(x, 2)).sum()/data["qty"].apply(lambda x:math.pow(x-avg, 2)).sum())

In [14]:
hat_svr = test_i.loc[:, ["qty", "predict", "day"]]
hat_svr.to_csv("2p3/2/svr.csv")
RSE_svr = getRSE(hat_svr)
RSE_svr

0.654334116210174

In [15]:
hat_rfr = test_i_rfr.loc[:, ["qty", "predict", "day"]]
hat_rfr.to_csv("2p3/2/rfr.csv")
RSE_rfr = getRSE(hat_rfr)
RSE_rfr

0.5729715336751021

In [16]:
hat_mlp = test_i_mlp.loc[:, ["qty", "predict", "day"]]
hat_mlp.to_csv("2p3/2/mlp.csv")
RSE_mlp = getRSE(hat_mlp)
RSE_mlp

0.4981346901772707

In [17]:
horizon = range(1, 8)
results = [hat_svr, hat_rfr, hat_mlp]
RES_results = []
for re in results:
    ls = []
    for i in horizon:
        ls.append(getRSE(re.loc[re["day"]%7==(i-1), :]))
    RES_results.append(ls)
RES_results

[[0.9387316004595031,
  0.5703268836642078,
  0.4889698649411448,
  0.45870236296358813,
  0.5068874684709798,
  0.800259898764541,
  0.8961485364406993],
 [0.4271570728043932,
  0.5623360804181009,
  0.38286829858977556,
  0.8515163894744833,
  0.6728927674953398,
  0.5544649461597351,
  0.8244904419166643],
 [0.5973192722491424,
  0.42640744553580623,
  0.32500128265371714,
  0.4621042894953252,
  0.5788238155258085,
  0.7071288008515509,
  0.6641957031767457]]

In [13]:
time_list

[[274.44811272621155, 28.80157208442688],
 [49.77377128601074, 101.10513091087341],
 [38.8939573764801, 30.435043334960938]]